Her importere jeg forskelige ting jeg bruger senere

In [2]:
import pandas as pd
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Først importeres results.csv som et dataframe som df 

derefter laves et nyt row og ved hjælp af en lambda funktion bestemes om en kamp har været, uafgjort, hjemmeholdet har vundet eller om udeholdet har vundet
så laves der endnu et row hvor resultatet af kampen bliver konverteret til et tal 0 hvis det bliver uafgjort, 1 hvis hjemmeholdet vinder og 2 hvis udeholdet vinder
derefter bliver home_team og away_team lagt sammen som combined_series
derefter findes alle de unikke lande i datasættet med .unique()
så laves der et dataframe countries hvor landene bliver gemt i
så laves der et nyt row hvor hvert land bliver givet et tal
derefter laves der et dictonery hvor landet og dets tal bliver gemt
så bruges .map() til at erstatte alle landene i datasættet med deres tal
så bliver datasættet modificeret så kun "home_team", "away_team" og "result" bliver gemt
til sidst bliver datasættet delt for senre at kunne teste modellen

In [3]:
df = pd.read_csv('results.csv')

df['winning_team'] = df.apply(lambda row: row['home_team'] if row['home_score'] > row['away_score'] else (row['away_team'] if row['home_score'] < row['away_score'] else 'Draw'), axis=1)

df = df[["home_team", "away_team", "winning_team"]]

df['result'] = [1 if row['winning_team'] == row['home_team'] else 
                2 if row['winning_team'] == row['away_team'] else 
                0
                for _, row in df.iterrows()]
combined_series = pd.concat([df['home_team'], df['away_team']])

unique_countries = combined_series.unique()

countries = pd.DataFrame({'unique_country': unique_countries})

countries['Number'] = range(1, len(countries) + 1)

country_to_number = dict(zip(countries['unique_country'], countries['Number']))


df['home_team'] = df['home_team'].map(country_to_number)
df['away_team'] = df['away_team'].map(country_to_number)

df = df[["home_team", "away_team", "result"]]
train, test = train_test_split(df, test_size=0.2, shuffle=True)

Her defineres features med home_team og away_team da det er udfra holdene at modellen skal gætte resultatet derefter gemmes X med features fra train datasættet, y defineres så med result istedet
det samme gøres så med X_test og y_test
herefter bliver modellen så lavet med dtree da vi bruger et decision tree til vores model
til sidst bliver decision tree'et så plottet

In [4]:
features = ['home_team', 'away_team']

X = train[features]
y = train['result']

X_test = test[features]
y_test = test['result']

dtree = DecisionTreeClassifier()
dtree = dtree.fit(X, y)

tree.plot_tree(dtree, feature_names=features) 

[Text(0.3654449646886326, 0.9852941176470589, 'away_team <= 51.5\ngini = 0.63\nsamples = 35809\nvalue = [8217, 17454, 10138]'),
 Text(0.13825815832515476, 0.9558823529411765, 'home_team <= 48.5\ngini = 0.65\nsamples = 12642\nvalue = [2975, 5268, 4399]'),
 Text(0.03838018734912222, 0.9264705882352942, 'away_team <= 2.5\ngini = 0.633\nsamples = 8167\nvalue = [1887, 3906, 2374]'),
 Text(0.002704793892477167, 0.8970588235294118, 'away_team <= 1.5\ngini = 0.642\nsamples = 621\nvalue = [147, 191, 283]'),
 Text(0.0009203938867474421, 0.8676470588235294, 'home_team <= 4.5\ngini = 0.641\nsamples = 277\nvalue = [56, 111, 110]'),
 Text(0.0003492403680938911, 0.8382352941176471, 'home_team <= 3.5\ngini = 0.632\nsamples = 128\nvalue = [28, 39, 61]'),
 Text(0.00023282691206259407, 0.8088235294117647, 'home_team <= 2.5\ngini = 0.655\nsamples = 89\nvalue = [22, 32, 35]'),
 Text(0.00011641345603129703, 0.7794117647058824, 'gini = 0.642\nsamples = 45\nvalue = [10, 20, 15]'),
 Text(0.0003492403680938911,

her prøves modellen så med hold 11 og 3
og derefter testes accuracy der får vi 49.55% hvilket er ret gdt når der er 3 muligheder i en fodboldkamp

In [5]:
print(dtree.predict([[11, 3]])) 
print(dtree.score(X_test, y_test))

[1]
0.4994973751815034


C:\Users\Alfred\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Her bruges pickle til at gemme modellen

In [6]:
import pickle
with open("data", "ab") as file:
    pickle.dump(dtree, file)
pickle.dump(country_to_number, open("country_to_number", "ab"))